In [1]:
import glob
import os
import numpy as np
from PIL import Image
import skimage
import skimage.io
import scipy.misc
import imageio
import warnings; warnings.simplefilter('ignore')

list_classes = [
 'Sony-NEX-7',
 'Motorola-X',
 'HTC-1-M7',
 'Samsung-Galaxy-Note3',
 'Motorola-Droid-Maxx',
 'iPhone-4s',
 'iPhone-6',
 'LG-Nexus-5x',
 'Samsung-Galaxy-S4',
 'Motorola-Nexus-6']

list_dict = {}
for i in range(10):
    key = list_classes[i]
    v = [0,0,0,0,0,0,0,0,0,0]
    v[i] = 1
    list_dict[key] = v
print(list_dict)

train_dir = '../input/train'
test_dir = '../input/test'
test_files = sorted(glob.glob(test_dir+'/*'))
train_files = sorted(glob.glob(train_dir+'/*/*'))
train_data_cnt = len(train_files)
BATCH_SIZE = 48
CROP_LEN = 112

def random_crop(im_array):
    # crop
    x_range = im_array.shape[0] - CROP_LEN
    y_range = im_array.shape[1] - CROP_LEN
    # print(x_range,y_range)
    a = np.random.randint(x_range)
    b = a + CROP_LEN
    c = np.random.randint(y_range)
    d = c + CROP_LEN
    new_im_array = im_array[a:b,c:d,:]
    return new_im_array

def center_crop(im_array):
    center_x = im_array.shape[0] // 2
    center_y = im_array.shape[1] // 2
    half_crop = CROP_LEN // 2
    a = center_x - half_crop
    b = a + CROP_LEN
    c = center_y - half_crop
    d = c + CROP_LEN
    new_im_array = im_array[a:b,c:d,:]
    return new_im_array
    

def random_manip(img,rnd):
    if rnd == 0:
        return img
    
    elif rnd == 1:
        # gamma 0.8
        new_img = skimage.exposure.adjust_gamma(img, gamma=0.8)
        return new_img
    
    elif rnd == 2:
        # gamma 1.2
        new_img = skimage.exposure.adjust_gamma(img, gamma=1.2)
        return new_img
    
    elif rnd == 3:
        # jpeg 70
        imageio.imwrite('/tmp/quality-70.jpg', img, quality=70)
        try:
            new_img = np.array(Image.open(('/tmp/quality-70.jpg')), dtype="uint8")
            os.remove('/tmp/quality-70.jpg')
            return new_img
        except:
            return img
    
    elif rnd == 4:
        # jpeg 90
        imageio.imwrite('/tmp/quality-90.jpg', img, quality=90)
        try:
            new_img = np.array(Image.open(('/tmp/quality-90.jpg')), dtype="uint8")
            os.remove('/tmp/quality-90.jpg')
            return new_img
        except:
            return img
    
    elif rnd == 5:
        # 2x of original image
        new_img = scipy.misc.imresize(img, 2.0, interp='bicubic')
        return center_crop(new_img)
    
    elif rnd == 6:
        new_img = scipy.misc.imresize(img, 1.5, interp='bicubic')
        return center_crop(new_img)
    
    elif rnd == 7:
        new_img = scipy.misc.imresize(img, 0.5, interp='bicubic')
        return new_img
    
    elif rnd == 8:
        new_img = scipy.misc.imresize(img, 0.8, interp='bicubic')
        return new_img
    
    else:
        return img

{'LG-Nexus-5x': [0, 0, 0, 0, 0, 0, 0, 1, 0, 0], 'Motorola-Nexus-6': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1], 'iPhone-4s': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0], 'Samsung-Galaxy-Note3': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0], 'HTC-1-M7': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0], 'Motorola-X': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0], 'iPhone-6': [0, 0, 0, 0, 0, 0, 1, 0, 0, 0], 'Sony-NEX-7': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'Motorola-Droid-Maxx': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0], 'Samsung-Galaxy-S4': [0, 0, 0, 0, 0, 0, 0, 0, 1, 0]}


In [2]:
def get_img(img_path, train_flag = True, fake_rnd = 0, rnd_range=16):
    # read img
    im_array = np.array(Image.open((img_path)), dtype="uint8")
    
    # train or valid
    if train_flag is True:
        # manip rnd
        manip_rnd = np.random.randint(rnd_range) # only half manip, 1 to 8 use manip
        #manip_rnd = 5
        #print(manip_rnd,im_array.shape)
        if manip_rnd < 7 or manip_rnd > 8 : 
            # no zoom in， 随机切出 224
            im_array = random_crop(im_array)
            # manip， 随机变化，如果放大则去中间部分
            final_img = random_manip(im_array, manip_rnd)
          
        else:
            # resize zoom out， 缩小
            im_array = random_manip(im_array, manip_rnd)
            # random crop on larger image， 随机切出 224
            final_img = random_crop(im_array)
        if final_img.shape[0]!=CROP_LEN or final_img.shape[1]!=CROP_LEN:
            print('train',manip_rnd,final_img.shape,img_path,im_array.shape)
    else:
        # center crop for valid data
        # manip rnd
        manip_rnd = fake_rnd
        if manip_rnd < 7 or manip_rnd > 8 :
            # no zoom in， 取中间部分
            im_array = center_crop(im_array)
            # manip, 随机变化，如果放大则取中间部分
            final_img = random_manip(im_array, manip_rnd)
        else:
            # resize zoom out, 先缩小图片
            im_array = random_manip(im_array, manip_rnd)
            # random crop on larger image, 取中间部分
            final_img = center_crop(im_array)
        if final_img.shape[0]!=CROP_LEN or final_img.shape[1]!=CROP_LEN:
            print('valid',manip_rnd,final_img.shape,img_path,im_array.shape)
    
    final_img = final_img/255.0
    return final_img

# 读取测试数据中间 112
def get_test_img(img_path):
    # read img
    im_array = np.array(Image.open((img_path)), dtype="uint8")
    final_img = center_crop(im_array)
    final_img = final_img/255.0
    return final_img

def get_y(img_path):
    n = img_path.split('/')[-2]
    return list_dict[n]

def data_gen(file_list, batch_size=BATCH_SIZE, train_flag = True, manip_rnd_range=16):
    curr_idx = 0
    data_cnt = len(file_list)
    fake_rnd = 0
    while True:
        if curr_idx + batch_size > data_cnt:
            start_idx = data_cnt-batch_size
            end_idx = data_cnt
            curr_idx = 0
            fake_rnd = 0
        else:
            start_idx = curr_idx
            end_idx = curr_idx + batch_size
            curr_idx += batch_size
        curr_fl = file_list[start_idx:end_idx]
        if train_flag is True:
            curr_x = np.array([get_img(p,train_flag,0,manip_rnd_range) for p in curr_fl],dtype='float32')
        else:
            # make validation data stable
            curr_x = []
            for p in curr_fl:
                tmp_img = get_img(p,train_flag,fake_rnd)
                curr_x.append(tmp_img)
                fake_rnd += 1
                fake_rnd = fake_rnd % manip_rnd_range
            curr_x = np.array(curr_x,dtype='float32')
        curr_y = np.array([get_y(p) for p in curr_fl])
        yield curr_x,curr_y

from sklearn.utils import shuffle
train_files = shuffle(train_files,random_state=42)

# 1/8 manip
train_gen_l = data_gen(train_files, BATCH_SIZE, True, 64)
valid_gen_l = data_gen(train_files, BATCH_SIZE, False, 64)

# 1/2 manip
train_gen = data_gen(train_files, BATCH_SIZE, True)
valid_gen = data_gen(train_files, BATCH_SIZE, False)
import math
train_step = math.ceil(train_data_cnt/BATCH_SIZE)
valid_step = train_step

print(train_step)
    

58


In [3]:
# def model
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from keras.optimizers import Adam
from keras import backend as K

Using TensorFlow backend.


In [4]:
model_p = 'best_xception_manip.h5'

if os.path.exists(model_p):
    model = load_model(model_p)
    K.set_value(model.optimizer.lr, 0.0001)
    print('load',model_p)
else:
    from keras.applications.xception import Xception
    model = Xception(input_shape=(CROP_LEN,CROP_LEN,3),include_top=True,weights=None,classes=10)
    print('get model')
    model.compile(loss='categorical_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])
    
    
model_chk = ModelCheckpoint(filepath=model_p, monitor='val_acc', save_best_only=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5, verbose=1,
                              patience=3, min_lr=0.00000001)

model.fit_generator(train_gen_l,
          steps_per_epoch = train_step,
          epochs=100,
          callbacks=[reduce_lr]
         )
model.save('less_manip_xception.h5')
K.set_value(model.optimizer.lr, 0.0001)
model.fit_generator(train_gen,
          steps_per_epoch = train_step,
          epochs=150,
          validation_data = valid_gen,
          validation_steps = valid_step,
          callbacks=[model_chk,reduce_lr]
         )

load best_xception_manip.h5
Epoch 1/100
58/58 [==============================] - 403s 7s/step - loss: 0.7764 - acc: 0.7435
Epoch 2/100
58/58 [==============================] - 419s 7s/step - loss: 0.7089 - acc: 0.7615
Epoch 3/100
58/58 [==============================] - 422s 7s/step - loss: 0.7506 - acc: 0.7471
Epoch 4/100
58/58 [==============================] - 410s 7s/step - loss: 0.7403 - acc: 0.7511
Epoch 5/100
58/58 [==============================] - 402s 7s/step - loss: 0.7308 - acc: 0.7511
Epoch 6/100
57/58 [============================>.] - ETA: 7s - loss: 0.7165 - acc: 0.7591 
Epoch 00006: reducing learning rate to 4.999999873689376e-05.
58/58 [==============================] - 410s 7s/step - loss: 0.7112 - acc: 0.7611
Epoch 7/100
58/58 [==============================] - 404s 7s/step - loss: 0.6838 - acc: 0.7622
Epoch 8/100
58/58 [==============================] - 408s 7s/step - loss: 0.6618 - acc: 0.7805
Epoch 9/100
58/58 [==============================] - 404s 7s/step - los

In [5]:
import pandas as pd
best_model = load_model(model_p)

print('load done')

load done


In [6]:
test_y = []
for img_p in test_files:
    tmp_x = get_test_img(img_p)
    tmp_y = best_model.predict(np.array([tmp_x]))[0]
    test_y.append(tmp_y)

test_y = np.array(test_y)
print(test_y[:5])
y_res = np.argmax(test_y,axis=1)
y_res = [list_classes[i] for i in y_res]
df = pd.read_csv('../input/sample_submission.csv')
print(df.head())
f_name = [p.split('/')[-1] for p in test_files]
df['fname'] = f_name
df['camera'] = y_res
print(df.head())

[[  2.35914234e-02   6.15372658e-02   1.90893537e-03   1.73851401e-02
    1.67276673e-02   1.37470081e-03   3.33489418e-01   1.45779476e-01
    4.28049713e-02   3.55401069e-01]
 [  1.12465341e-06   5.66834249e-02   7.63957053e-02   2.44883075e-02
    1.09239727e-01   3.49715382e-01   8.04397687e-02   2.36662969e-01
    4.13166210e-02   2.50569116e-02]
 [  2.69867323e-05   5.11605619e-03   1.24755572e-03   6.40310172e-05
    2.60226894e-04   8.69239448e-04   9.89385307e-01   1.78651186e-04
    1.99970556e-03   8.52165103e-04]
 [  6.93342145e-07   4.87816073e-02   9.62113903e-04   7.34749213e-02
    9.96974195e-05   6.96693533e-05   7.15067319e-04   3.77595127e-02
    6.21810436e-01   2.16326296e-01]
 [  1.36327671e-05   1.99300930e-01   7.72509500e-02   1.96064934e-02
    6.26200065e-03   1.99396890e-02   2.47989204e-02   5.64496994e-01
    7.53870010e-02   1.29433218e-02]]
                   fname    camera
0  img_0002a04_manip.tif  iPhone-6
1  img_001e31c_unalt.tif  iPhone-6
2  img_00

In [7]:
df.to_csv('../results/xception_112_manip.csv',index=False)

In [8]:
# TTA
def get_tta_img(img_path):
    im_array = np.array(Image.open((img_path)), dtype="uint8")
    im_array = im_array/255.0
    img_list = []
    img_list.append(center_crop(im_array))
    for i in range(5):
        for j in range(5):
            a = i*100
            b = a + 112
            c = j*100
            d = c + 112
            img_list.append(im_array[a:b,c:d,:])
    return np.array(img_list)
tta_img = get_tta_img(test_files[0])
print(tta_img.shape)
tta_res = best_model.predict(tta_img)
print(tta_res.shape)
print(np.sum(tta_res,axis=0))

test_y = []
for img_p in test_files:
    tmp_x = get_tta_img(img_p)
    tmp_y = best_model.predict(tmp_x)
    tmp_y = np.sum(tmp_y,axis=0)
    test_y.append(tmp_y)

test_y = np.array(test_y)
print(test_y[:5])

y_res = np.argmax(test_y,axis=1)
y_res = [list_classes[i] for i in y_res]
df = pd.read_csv('../input/sample_submission.csv')
f_name = [p.split('/')[-1] for p in test_files]
df['fname'] = f_name
df['camera'] = y_res
print(df.head())
df.to_csv('../results/xception_manip_tta.csv',index=False)

(26, 112, 112, 3)
(26, 10)
[ 0.11134116  1.92894149  0.05166588  1.87150943  0.37292564  0.02033568
  2.62859917  9.46186924  0.70615876  8.84665394]
[[  1.11341164e-01   1.92894149e+00   5.16658761e-02   1.87150943e+00
    3.72925639e-01   2.03356836e-02   2.62859917e+00   9.46186924e+00
    7.06158757e-01   8.84665394e+00]
 [  3.72949094e-02   3.16932464e+00   1.44667614e+00   6.84789598e-01
    4.95884705e+00   4.47738838e+00   1.82153094e+00   5.89711905e+00
    6.58377111e-01   2.84865212e+00]
 [  2.38415644e-01   7.08163142e-01   1.18258500e+00   8.41547996e-02
    5.65323293e-01   2.01076254e-01   2.06116753e+01   2.14659643e+00
    1.81933954e-01   8.00731033e-02]
 [  1.87660810e-02   7.76696146e-01   6.14207387e-01   1.52527869e+00
    2.33495031e-02   1.91089418e-02   1.44752607e-01   8.30281675e-01
    1.92263451e+01   2.82121277e+00]
 [  3.04921716e-03   1.49780550e+01   2.59435558e+00   2.59006047e+00
    2.68160906e-02   1.10069580e-01   9.46041644e-02   3.61300159e+00
  